In [1]:
# Training Data for implementation of Decision Tree

training_data = [
    ['Sunny', 'Hot', 'High', 'Weak', 'No'],
    ['Sunny', 'Hot', 'High', 'Strong', 'No'],
    ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
    ['Sunny', 'Mild', 'High', 'Weak', 'No'],
    ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
    ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
    ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
    ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Rain', 'Mild', 'High', 'Strong', 'No'],
]

# Column Labels
header = ['Outlook', 'Temp', 'Humidity', 'Wind', 'Played Football']

In [2]:
# Extracting Unique Values

def unique_vals(rows, col):
    return set([row[col] for row in rows])

unique_vals(training_data,0)

{'Overcast', 'Rain', 'Sunny'}

In [3]:
# Counting 'Yes' and 'No'(Output Frequency) for the given Dataset

def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

class_counts(training_data)

{'No': 5, 'Yes': 9}

In [4]:
# Questions are basically the points on which the Decision Tree should be built

class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    # This function matches the given string with the actual string      
    def match(self, example):
        val = example[self.column]
        return val == self.value
    
    # This function is just used for interpretable output
    def __repr__(self):
        condition = '=='
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
    
q = Question(0, 'Overcast') 
print(q)
print(q.match(training_data[3]))

Is Outlook == Overcast?
False


In [5]:
# Partition is basically splitting in True rows(Positive Answer in response to the Question) 
# and False rows(Negative Answer in response to the Question)

def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

true_rows, false_rows = partition(training_data, Question(0, 'Sunny'))
print(true_rows,"\n",false_rows)

[['Sunny', 'Hot', 'High', 'Weak', 'No'], ['Sunny', 'Hot', 'High', 'Strong', 'No'], ['Sunny', 'Mild', 'High', 'Weak', 'No'], ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'], ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes']] 
 [['Overcast', 'Hot', 'High', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'], ['Rain', 'Cool', 'Normal', 'Strong', 'No'], ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'], ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'], ['Overcast', 'Mild', 'High', 'Strong', 'Yes'], ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'], ['Rain', 'Mild', 'High', 'Strong', 'No']]


In [6]:
# Calculation of Gini Impurity Index 

def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [7]:
no_mixing = [['Overcast'],
              ['Overcast']]

print(gini(no_mixing))

mixing = [['Overcast'],
              ['Rain']]
print(gini(mixing))

0.0
0.5


In [8]:
# Calculating Info-Gain for the features

def info_gain(left, right, current_uncertainty):
    p = float(len(left) / (len(left) + len(right)))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)

curr_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Sunny'))
info_gain(true_rows, false_rows, curr_uncertainty)

0.0655328798185941

In [9]:
# Finding the best split depending on the Gain and Question.

def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
                
    return best_gain, best_question

best_gain, best_question = find_best_split(training_data)
best_question

Is Outlook == Overcast?

In [10]:
# Leaf node classifies the data.

class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [11]:
# A Decision Node asks a question. This holds a reference to the question, and to the two child nodes.

class Decision_Node:
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [12]:
# Building the Decision Tree.

def build_tree(rows):
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows, false_rows = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [13]:
# Printing the Decision Tree.

def print_tree(node, spacing=''):
    
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    print(spacing + str(node.question))
    
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + " ")
    
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + " ")

In [14]:
my_tree = build_tree(training_data)

In [15]:
print_tree(my_tree)

Is Outlook == Overcast?
--> True:
 Predict {'Yes': 4}
--> False:
 Is Humidity == Normal?
 --> True:
  Is Wind == Strong?
  --> True:
   Is Temp == Cool?
   --> True:
    Predict {'No': 1}
   --> False:
    Predict {'Yes': 1}
  --> False:
   Predict {'Yes': 3}
 --> False:
  Is Outlook == Rain?
  --> True:
   Is Wind == Strong?
   --> True:
    Predict {'No': 1}
   --> False:
    Predict {'Yes': 1}
  --> False:
   Predict {'No': 3}


In [16]:
# Classifying the Leaf Nodes by Traversing the Decision Tree from Root Node to the Leaf Node.

def classify(row, node):
    
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    
classify(training_data[0], my_tree)

{'No': 3}

In [29]:
# Printing the predictions at a leaf.

def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

print_leaf(classify(training_data[0], my_tree))

{'No': '100%'}

In [30]:
# Validation Dataset

testing_data = [
    ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
    ['Sunny', 'Hot', 'Normal', 'Weak', 'Yes'],
    ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes']
]

In [31]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: No. Predicted: {'No': '100%'}
Actual: Yes. Predicted: {'Yes': '100%'}
Actual: Yes. Predicted: {'Yes': '100%'}
